<img src='https://www.icos-cp.eu/sites/default/files/2017-11/ICOS_CP_logo.png' width=400 align=right>

# ICOS Carbon Portal Python Libraries


# Direct access to spatiotemporal datasets without the need to download big data

Also demonstrating the use of xarray for plotting and pickle for storing intermediate results

In [ ]:
# import and read data
from icoscp_core.icos import meta,data,auth,bootstrap
import xarray as xr
import pandas as pd
import numpy as np
import datetime as dt
import os
import pickle
import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
from IPython.core.display import HTML
import warnings
warnings.filterwarnings('ignore')

cwd = os.getcwd()
cwd

This little helper function gives us direct access to a netcdf data object in the ICOS data store through and returns a xarray data object

In [ ]:
def xr_nc_CP(uri : str):
    '''
    Little helper function
    At the ICOS Jupyter instances we can access (read only) all data files directly through their 
    hash sums. These hases are included in the data object URI. In this function we access the 
    spatiotemporal netcdf files, where we know that their path in the ICOS Jupyterhub is 
    /data/dataAppStorage/netcdf. The netcdf timeseries (for example for Obspack) are in path
    /data/dataAppStorage/netcdfTimeSeries. This function returns an xarray data structure
    for a netcdf file with the given URI
    '''
    dobj_meta = meta.get_dobj_meta(uri)
    ncpath='/data/dataAppStorage/netcdf/'+dobj_meta.pid[6:] # remove the handle prefix ('11676/')
    data=xr.open_dataset(ncpath)
    return data
    

We define the datatypes for the Carbontracker High resolution flux data we want to use and the year for which we want to access the data, and show the flux timeseries in a point [lat, lon]

In [ ]:
fire = 'http://meta.icos-cp.eu/resources/cpmeta/fireEmissionModelResults'
ocean = 'http://meta.icos-cp.eu/resources/cpmeta/oceanicFluxModelResults'
anthro = 'http://meta.icos-cp.eu/resources/cpmeta/anthropogenicEmissionModelResults'
nep = 'http://meta.icos-cp.eu/resources/cpmeta/biosphericModelResults'   

datatypes = [ocean,anthro,nep,fire]
ouryear = 2022
ourlat = 53.0
ourlon = 5.0

## Show a map of average modelled antropogenic CO2 emissions for a the chosen year

In [ ]:
# Get a list of data objects of the CTHRE type for anthropogenic emissions and select those 
# for the year we are interested in; list the description and the variables contained in this datatype

latest = meta.list_data_objects(datatype=anthro,limit=10000)
selection = [do for do in latest if (do.time_start.year==2022) and not('persector' in do.filename)]
dobj_meta = meta.get_dobj_meta(selection[0].uri)
print(dobj_meta.specificInfo.description)
print('Variables:')
[print(do.label,'=',do.valueType.self.label,'['+do.valueType.unit+']') for do in dobj_meta.specificInfo.variables]

In [ ]:
# read the netcdf files for the selected period and calculate the average
# the result is saved in a pickle file to save 1 minute per netcdf file when rerunning

show=False # is show is True then we plot a map for each individual file
if not(os.path.isfile('data/aaver.pkl')): 
    i=0
    for do in selection:
        print(i)
        anthro=xr_nc_CP(do.uri)
        if i==0:
            aaver=anthro.combustion.mean(dim="time",skipna=True)+anthro.cement.mean(dim="time",skipna=True)
        else:                       
            aaver=aaver+anthro.combustion.mean(dim="time",skipna=True)+anthro.cement.mean(dim="time",skipna=True)
        if show:
            fig = plt.figure(figsize=(5, 5))
            aaver.plot(levels=[1e-8,2e-8,5e-8,1e-7,2e-7,5e-7,1e-6,2e-6,5e-6,1e-5,2e-5,5e-5])
            plt.show()
        anthro.close()    
        i=i+1
    aaver=aaver/len(selection)
    with open('data/aaver.pkl', 'wb') as f:
        pickle.dump(aaver,f,protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('data/aaver.pkl', 'rb') as f:
    aaver=pickle.load(f)

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection=ccrs.Robinson())
ax.coastlines(resolution="10m")
plot=aaver.plot(levels=[1e-8,2e-8,5e-8,1e-7,2e-7,5e-7,1e-6,2e-6,5e-6,1e-5,2e-5,5e-5],
           cmap=plt.cm.coolwarm,
           transform=ccrs.PlateCarree(),
           cbar_kwargs={"shrink": 0.6})
plt.title('Anthropogenic emissions')
plt.show()

## Show a timeseries plot of modelled Anthropogenic combustion fluxes for a point on the map

In [ ]:
selection = [do for do in latest if (do.time_start.year==ouryear) & (do.time_start.month==6)]
anthro=xr_nc_CP(selection[0])
anthroNL=anthro.combustion.sel(latitude=ourlat, longitude=ourlon, method="nearest")

fig,[ax1,ax2]=plt.subplots(1,2,figsize=(15,5))
ax1.plot(anthroNL)
ax1.set_title('June')

selection = [do for do in latest if (do.time_start.year==ouryear) & (do.time_start.month==12)]
anthro=xr_nc_CP(selection[0])
anthroNL=anthro.combustion.sel(latitude=53.0, longitude=5.0, method="nearest")
ax2.plot(anthroNL)
ax2.set_title('December')
ax2.set_ylim(ax1.get_ylim())
plt.show()

## Show a map of average modelled NEE for a the chosen year

In [ ]:
latest = meta.list_data_objects(datatype=nep,limit=10000)
selection = [do for do in latest if (do.time_start.year==ouryear)]
dobj_meta = meta.get_dobj_meta(selection[0].uri)
print(dobj_meta.specificInfo.description)
print('Variables:')
[print(do.label,'=',do.valueType.self.label,'['+do.valueType.unit+']') for do in dobj_meta.specificInfo.variables]

In [ ]:
# read the netcdf files for the selected period and calculate the average
# the result is saved in a pickle file to save 1 minute per netcdf file when rerunning
show=False
if not(os.path.isfile('data/nepaver.pkl')): 
    i=0
    for do in selection:
        print(i)
        nep=xr_nc_CP(do.uri)
        if i==0:
            nepaver=nep.mean(dim="time",skipna=True)
        else:                       
            nepaver=nepaver+nep.mean(dim="time",skipna=True)
        if show:
            fig = plt.figure(figsize=(5, 5))
            aaver.plot(levels=[-5e-7,-2e-7,-1e-7,-5e-8,-2e-8,-1e-8,1e-8,2e-8,5e-8,1e-7,2e-7,5e-7])
            plt.show()
        nep.close()    
        i=i+1
    nepaver=nepaver/len(selection)
    with open('data/nepaver.pkl', 'wb') as f:
        pickle.dump(nepaver,f,protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('data/nepaver.pkl', 'rb') as f:
    nepaver=pickle.load(f)

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection=ccrs.Robinson())
ax.coastlines(resolution="10m")
plot=nepaver.nep.plot(cmap=plt.cm.coolwarm,
                      transform=ccrs.PlateCarree(),
                      cbar_kwargs={"shrink": 0.6})
plt.title('NEP')
plt.show()

## Show a timeseries plot of modelled NEE for a point on the map

In [ ]:
selection = [do for do in latest if (do.time_start.year==ouryear) & (do.time_start.month==6)]
nep=xr_nc_CP(selection[0])
nepNL=nep.sel(latitude=ourlat, longitude=ourlon, method="nearest")

fig,[ax1,ax2]=plt.subplots(1,2,figsize=(10,5))
ax1.plot(nepNL.nep)
ax1.set_title('June')

selection = [do for do in latest if (do.time_start.year==ouryear) & (do.time_start.month==12)]
nep=xr_nc_CP(selection[0])
nepNL=nep.sel(latitude=53.0, longitude=5.0, method="nearest")
ax2.plot(nepNL.nep)
ax2.set_title('December')
ax2.set_ylim(ax1.get_ylim())
plt.show()

## Show a map of average modelled Ocean fluxes for a the chosen year

In [ ]:
latest = meta.list_data_objects(datatype=ocean,limit=10000)
selection = [do for do in latest if (do.time_start.year==ouryear) and not('persector' in do.filename)]
dobj_meta = meta.get_dobj_meta(selection[0].uri)
print(dobj_meta.specificInfo.description)
print('Variables:')
[print(do.label,'=',do.valueType.self.label,'['+do.valueType.unit+']') for do in dobj_meta.specificInfo.variables]

In [ ]:
# read the netcdf files for the selected period and calculate the average
# the result is saved in a pickle file to save 1 minute per netcdf file when rerunning
show=False
if not(os.path.isfile('data/oceaver.pkl')): 
    i=0
    for do in selection:
        print(i)
        oce=xr_nc_CP(do.uri)
        if i==0:
            oceaver=oce.ocean.mean(dim="time",skipna=True)
        else:                       
            oceaver=oceaver+oce.ocean.mean(dim="time",skipna=True)
        if show:
            fig = plt.figure(figsize=(5, 5))
            ocever.plot(levels=[1e-8,2e-8,5e-8,1e-7,2e-7,5e-7,1e-6,2e-6,5e-6,1e-5,2e-5,5e-5])
            plt.show()
        oce.close()    
        i=i+1
    oceaver=oceaver/len(selection)
    with open('data/oceaver.pkl', 'wb') as f:
        pickle.dump(oceaver,f,protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('data/oceaver.pkl', 'rb') as f:
    oceaver=pickle.load(f)

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection=ccrs.Robinson())
ax.coastlines(resolution="10m")
plot=oceaver.plot(cmap=plt.cm.coolwarm,
                  transform=ccrs.PlateCarree(),
                  cbar_kwargs={"shrink": 0.6})
plt.title('Ocean fluxes')
plt.show()

## Show a map of average CO2 fire emissions for the given year

In [ ]:
latest = meta.list_data_objects(datatype=fire,limit=10000)
selection = [do for do in latest if (do.time_start.year==ouryear) and not('persector' in do.filename)]
dobj_meta = meta.get_dobj_meta(selection[0].uri)
print(dobj_meta.specificInfo.description)
print('Variables:')
[print(do.label,'=',do.valueType.self.label,'['+do.valueType.unit+']') for do in dobj_meta.specificInfo.variables]

In [ ]:
# read the netcdf files for the selected period and calculate the average
# the result is saved in a pickle file to save 1 minute per netcdf file when rerunning
show=False
if not(os.path.isfile('data/fireaver.pkl')): 
    i=0
    for do in selection:
        print(i)
        fire=xr_nc_CP(do.uri)
        next=fire.fire.mean(dim="time",skipna=True)
        if i==0:
            res=next
        else:                       
            res=res+next
        if show:
            fig = plt.figure(figsize=(5, 5))
            res.plot(levels=[1e-8,2e-8,5e-8,1e-7,2e-7,5e-7,1e-6,2e-6,5e-6,1e-5,2e-5,5e-5])
            plt.show()
        fire.close()    
        i=i+1
    fireaver=res/len(selection)
    with open('data/fireaver.pkl', 'wb') as f:
        pickle.dump(fireaver,f,protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('data/fireaver.pkl', 'rb') as f:
    fireaver=pickle.load(f)

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.axes(projection=ccrs.Robinson())
ax.coastlines(resolution="10m")
plot=fireaver.plot(levels=[1e-8,2e-8,5e-8,1e-7,2e-7,5e-7,1e-6,2e-6,5e-6,1e-5,2e-5,5e-5],
                  cmap=plt.cm.coolwarm,
                  transform=ccrs.PlateCarree(),
                  cbar_kwargs={"shrink": 0.6})
plt.title('Fire fluxes')
plt.show()